In [ ]:
from aoc2024 import load_example, load_input

In [ ]:
data = load_input(15)
data

In [ ]:
map_str, moves_str = data.split("\n\n", maxsplit=1)
moves_str = moves_str.replace("\n", "")
map_str, moves_str

In [ ]:
from enum import Enum
class Direction(Enum):
    UP = (-1, 0)
    DOWN = (1, 0)
    LEFT = (0, -1)
    RIGHT = (0, 1)


    @staticmethod
    def from_symbol(symbol: str):
        return {
            "^": Direction.UP,
            "v": Direction.DOWN,
            "<": Direction.LEFT,
            ">": Direction.RIGHT,
        }[symbol]
    
    def __str__(self):
        return {
            "UP": "^",
            "DOWN": "v",
            "LEFT": "<",
            "RIGHT": ">"
        }[self.name]

In [ ]:
moves = [Direction.from_symbol(c) for c in moves_str]
moves

In [ ]:
map_str.splitlines()

In [ ]:
from enum import Enum
class TileObject(Enum):
    EMPTY = "."
    WALL = "#"
    BOX = "O"
    ROBOT = "@"
    BOX_LEFT = "["
    BOX_RIGHT = "]"

    def __str__(self):
        return self.value

In [ ]:
def parse_map(map_str: str):
    map_rows = map_str.strip().splitlines()
    tiles = [
        [TileObject(c) for c in row]
        for row in map_rows
    ]
    return tiles

In [ ]:
tiles = parse_map(map_str)
tiles

In [ ]:
def print_map(tiles: list[list[TileObject]]):
    res = "\n".join(["".join([str(x) for x in row]) for row in tiles])
    print(res)

In [ ]:
def find_robot(tiles: list[list[TileObject]]) -> tuple[int, int]:
    for y, row in enumerate(tiles):
        for x, tile_object in enumerate(row):
            if tile_object == TileObject.ROBOT:
                return (y, x)
    return None

In [ ]:
def move(position: tuple[int, int], direction: Direction):
    y, x = position
    
    direction_y, direction_x = direction.value
    return y + direction_y, x + direction_x

In [ ]:
def move_object(tiles: list[list[TileObject]], position_from: tuple[int, int], direction: Direction) -> bool:
    y_from, x_from = position_from
    y_to, x_to = move(position_from, direction)


    object_to = tiles[y_to][x_to]
    object_from = tiles[y_from][x_from]

    if object_to == TileObject.WALL:
        return False
    
    elif object_to == TileObject.EMPTY:
        tiles[y_to][x_to] = object_from
        tiles[y_from][x_from] = TileObject.EMPTY
        return True
    
    elif object_to == TileObject.BOX:
        can_move_next_tile = move_object(tiles, (y_to, x_to), direction)

        if can_move_next_tile:
            tiles[y_to][x_to] = object_from
            tiles[y_from][x_from] = TileObject.EMPTY
            return True
        else:
            return False
    
    else:
        raise ValueError("Shouldn't move robot")


def process_move(tiles: list[list[TileObject]], position_from: tuple[int, int], direction: Direction):
    was_moved = move_object(tiles, position_from, direction)
    if was_moved:
        return move(position_from, direction)
    else:
        return position_from


In [ ]:
robot_position = find_robot(tiles)

for direction in moves:
    robot_position = process_move(tiles, robot_position, direction)

In [ ]:
print_map(tiles)

In [ ]:
def gps(y: int, x: int) -> int:
    return y * 100 + x

In [ ]:
def sum_gps(tiles: list[list[TileObject]]) -> int:
    res = 0
    for y, row in enumerate(tiles):
        for x, tile_object in enumerate(row):
            if tile_object == TileObject.BOX:
                res += gps(y, x)
    return res

In [ ]:
sum_gps(tiles)

## Part 2

In [ ]:
def can_move_object(tiles: list[list[TileObject]], position_from: tuple[int, int], direction: Direction, check_other: bool=True) -> bool:
    y_from, x_from = position_from
    y_to, x_to = move(position_from, direction)

    object_to = tiles[y_to][x_to]
    object_from = tiles[y_from][x_from]

    if object_from == TileObject.BOX_RIGHT and direction == Direction.LEFT:
        pos = move(position_from, Direction.LEFT)
        return can_move_object(tiles, pos, direction, check_other=False)
    
    if object_from == TileObject.BOX_LEFT and direction == Direction.RIGHT:
        pos = move(position_from, Direction.RIGHT)
        return can_move_object(tiles, pos, direction, check_other=False)

    if check_other and (object_from == TileObject.BOX_LEFT or object_from == TileObject.BOX_RIGHT):
        if object_from == TileObject.BOX_LEFT:
            other_pos = move(position_from, Direction.RIGHT)
        else:
            other_pos = move(position_from, Direction.LEFT)
        
        return can_move_object(tiles, position_from, direction, check_other=False) and can_move_object(tiles, other_pos, direction, check_other=False)

    if object_to == TileObject.WALL:
        return False
    
    elif object_to == TileObject.EMPTY:
        return True
    
    elif object_to == TileObject.BOX or object_to == TileObject.BOX_LEFT or object_to == TileObject.BOX_RIGHT:
        can_move_next_tile = can_move_object(tiles, (y_to, x_to), direction)

        if can_move_next_tile:
            return True
        else:
            return False
    
    else:
        raise ValueError("Shouldn't move robot")

In [ ]:
def move_object_2(tiles: list[list[TileObject]], position_from: tuple[int, int], direction: Direction, check_other=True):
    y_from, x_from = position_from
    y_to, x_to = move(position_from, direction)

    object_to = tiles[y_to][x_to]
    object_from = tiles[y_from][x_from]


    if object_from == TileObject.BOX_LEFT and direction == Direction.RIGHT:
        other_pos = move(position_from, Direction.RIGHT)
        move_object_2(tiles, other_pos, direction, False)
        
        tiles[y_to][x_to] = object_from
        tiles[y_from][x_from] = TileObject.EMPTY
        object_to = tiles[y_to][x_to]

    elif object_from == TileObject.BOX_RIGHT and direction == Direction.LEFT:
        other_pos = move(position_from, Direction.LEFT)
        move_object_2(tiles, other_pos, direction, False)
        
        tiles[y_to][x_to] = object_from
        tiles[y_from][x_from] = TileObject.EMPTY
        object_to = tiles[y_to][x_to]
    
    elif check_other and (object_from == TileObject.BOX_RIGHT or object_from == TileObject.BOX_LEFT):
        if object_from == TileObject.BOX_RIGHT:
            other_pos = move(position_from, Direction.LEFT)
        else:
            other_pos = move(position_from, Direction.RIGHT)

        move_object_2(tiles, other_pos, direction, check_other=False)
        move_object_2(tiles, position_from, direction, check_other=False)
    
    elif object_to == TileObject.EMPTY:
        tiles[y_to][x_to] = object_from
        tiles[y_from][x_from] = TileObject.EMPTY
    
    elif object_to == TileObject.BOX_LEFT or object_to == TileObject.BOX_RIGHT:
        move_object_2(tiles, (y_to, x_to), direction)
        
        tiles[y_to][x_to] = object_from
        tiles[y_from][x_from] = TileObject.EMPTY


In [ ]:
def resize_map(tiles: list[list[TileObject]]) -> list[list[TileObject]]:
    res = []

    for row in tiles:
        new_row = []
        for obj in row:
            print(obj.name)
            if obj.name == "BOX":
                new_row.append(TileObject.BOX_LEFT)
                new_row.append(TileObject.BOX_RIGHT)
            elif obj.name == "ROBOT":
                new_row.append(TileObject.ROBOT)
                new_row.append(TileObject.EMPTY)
            else:
                new_row.append(obj)
                new_row.append(obj)
        res.append(new_row)
    return res

In [ ]:
resized_map = resize_map(tiles)

In [ ]:
print_map(resized_map)

In [ ]:
robot_position = find_robot(resized_map)
robot_position

In [ ]:
direction = Direction.DOWN

In [ ]:

for direction in moves:
    print(direction)
    can_move = can_move_object(resized_map, robot_position, direction)
    if can_move:
        move_object_2(resized_map, robot_position, direction)
        robot_position = move(robot_position, direction)
    # print()
    # print()
    # print_map(resized_map)
print_map(resized_map)

In [ ]:
it = iter(moves)

In [ ]:
[str(m) for m in moves]


In [ ]:
direction = next(it)
print(direction)
can_move = can_move_object(resized_map, robot_position, direction)
print(can_move)

if can_move:
    move_object_2(resized_map, robot_position, direction)
    robot_position = move(robot_position, direction)
print()
print()
print_map(resized_map)

In [ ]:
def sum_gps2(tiles: list[list[TileObject]]) -> int:
    res = 0
    for y, row in enumerate(tiles):
        for x, tile_object in enumerate(row):
            if tile_object == TileObject.BOX_LEFT:
                res += gps(y, x)
    return res

In [ ]:
sum_gps2(resized_map)